## Assignment 2

Generate random data from a predefined linear regression model where $−5 ≤ β_i ≤ 5$ $i =
0, . . . , K$ with at least $K = 100$ independent variables $X = (X_1, X_2, . . . , X_K)$ and $n = 1000$
observations. We seek to adjust a multiple linear regression model to explain variable Y as a
function of the other variables X, i.e., $(Y = β
_0X + \epsilon)$ where $β = (β_0, β_1, . . . , β_K)$, by using
“Ridge Regression”:

\begin{align*}
&\min_{\beta} ||y − Xβ||_2^2 + ρ||β||_2^2
\\
\end{align*}

where ρ is a parameter (consider it fixed to a given value, e.g., ρ = 1).


#### a)  Estimate the value of the regression coefficients by implementing the analytical solution.

In [2]:
%matplotlib inline

import numpy as np
import time
import matplotlib.pyplot as plt
from numpy.linalg import inv
np.random.seed(1234)

K = 100
beta = np.random.randint(-5,5,size=([K+1,1]));
ntrain = 1000
ntest = 100
n = ntrain + ntest

X0 = np.ones([n,1]) # the first column has all values equal to one for the coefficients of beta_0
X1_s = np.random.uniform(0,100,([n,K]))
X_s = np.concatenate([X0, X1_s],axis=1)

## Values for the normal errors

error_s = np.random.normal(0,1,(n,1))

## Values for the y's

Y_s = np.dot(X_s,beta) + error_s

X = X_s[0:ntrain,:]
Y = Y_s[0:ntrain]

X_t = X_s[(ntrain+1):n,:]
Y_t = Y_s[(ntrain+1):n]

## Value of the regularization parameter
rho = 1
I = np.identity(K+1)

time_start = time.process_time()

beta_exact = np.dot(np.dot(inv(np.dot((X.T),X)+rho*I),X.T),Y)                   
                   

time_elapsed = (time.process_time() - time_start)

## Print the results

print('Values of the (exact) Ridge coefficients:')
for i in range(K+1):
    print('beta %3d %7.3f' %(i,beta_exact[i]))
print('Elapsed time = %8.5f' %(time_elapsed))

Values of the (exact) Ridge coefficients:
beta   0  -1.209
beta   1   0.999
beta   2  -0.001
beta   3  -1.002
beta   4   2.999
beta   5   4.001
beta   6  -4.002
beta   7   2.000
beta   8   3.999
beta   9   0.998
beta  10   3.001
beta  11  -4.998
beta  12   0.000
beta  13  -5.000
beta  14   4.000
beta  15   1.001
beta  16  -3.001
beta  17  -5.001
beta  18  -0.001
beta  19  -2.999
beta  20   1.001
beta  21  -1.998
beta  22   2.000
beta  23  -5.000
beta  24   3.999
beta  25  -5.000
beta  26  -2.001
beta  27  -3.002
beta  28  -1.999
beta  29  -3.998
beta  30  -2.001
beta  31  -4.000
beta  32  -2.001
beta  33   2.000
beta  34  -4.000
beta  35   2.001
beta  36  -1.000
beta  37  -5.001
beta  38   0.002
beta  39  -3.999
beta  40  -0.001
beta  41   4.001
beta  42   3.999
beta  43  -1.001
beta  44  -5.000
beta  45   4.000
beta  46   2.999
beta  47   2.998
beta  48   0.999
beta  49   3.000
beta  50   1.001
beta  51  -1.999
beta  52  -4.000
beta  53  -3.000
beta  54  -0.000
beta  55  -2.998
beta  

#### b) Estimate the value of the regression coefficients by using the function minimize from the Python module $Scipy.optimize$. Try at least four available solvers and compare their performance in terms of number of iterations, number of function, gradient and hessian evaluations as well as total computational time.

In [4]:
 from scipy.optimize import minimize
from numpy.linalg import norm

# Definition of the optimization problem
rho = 1

def ridge_reg(beta_ridge,X,Y, rho):
    beta_ridge = np.matrix(beta_ridge)
   # print("Tamano de X %d " % len(X))
    z = Y - np.dot(X,beta_ridge.T)
   # print("Tamano de Z %d " % len(z))
    val = np.dot(z.T,z) + rho*np.power(np.linalg.norm(beta_ridge),2)
    #val = np.dot(z.T,z) + np.dot(beta_ridge,beta_ridge.T)
    return val 
                   
# Define the first and second derivatives of the problem

def ridge_reg_der(beta_ridge,X,Y, rho):
    beta_ridge = np.matrix(beta_ridge)
    pp=-2*np.dot((Y-np.dot(X,beta_ridge.T)).T,X)+2*rho*beta_ridge
    aa=np.squeeze(np.asarray(pp))
    return aa

#Hessian

def ridge_hess(beta_ridge,X,Y, rho):
    ss = 2*np.dot(np.transpose(X),X)+2*rho*np.identity(101)
    return ss
 
beta_0 = np.zeros(K+1)
                 
## We solve the optimization algorithm using a derivative-free method   
rho = 1                  
time_start = time.process_time()

res = minimize(ridge_reg, beta_0, args=(X, Y, rho), method='Nelder-Mead', options={'disp': True,'xtol': 1e-10})
                   
time_elapsed = (time.process_time() - time_start) 
                   
print('\nValues of the Ridge coefficients obtained with Nelder-Mead:')
for i in range(K+1):
    print('beta %3d %7.3f' %(i,res.x[i]))
print('Elapsed time = %8.5f' %(time_elapsed)) 

err_val_1 = np.linalg.norm(np.transpose(beta_exact)-res.x,ord=2)/np.linalg.norm(np.transpose(beta_exact),ord=2)
print('\nError in values of coefficients = %8.4f' %err_val_1)
                   
## Using a variant of Newton's method  
                   
time_start2 = time.process_time()                   
                   
res2 = minimize(ridge_reg, beta_0, args=(X, Y, rho), method='Newton-CG', jac=ridge_reg_der, hess=ridge_hess, options={'disp': True})

time_elapsed2 = (time.process_time() - time_start2)

print('\nValues of the Ridge coefficients obtained with Newton-CG:')
for i in range(K+1):
    print('beta %3d %7.3f' %(i,res.x[i]))
print('Elapsed time = %8.5f' %(time_elapsed2))
                   
err_val_2 = np.linalg.norm(np.transpose(beta_exact)-res.x,ord=2)/np.linalg.norm(np.transpose(beta_exact),ord=2)
print('\nError in values of coefficients = %8.4f' %err_val_2)

## Using a Quasi-Newton's method     
                   
time_start3 = time.process_time()                   
                                      
res3 = minimize(ridge_reg, beta_0, args=(X, Y, rho), method='BFGS', jac=ridge_reg_der, options={'disp': True})
                   
time_elapsed3 = (time.process_time() - time_start3) 
                   
print('\nValues of the Ridge coefficients obtained with BFGS:')
for i in range(K+1):
    print('beta %3d %7.3f' %(i,res.x[i]))
print('Elapsed time = %8.5f' %(time_elapsed3))

err_val_3 = np.linalg.norm(np.transpose(beta_exact)-res.x,ord=2)/np.linalg.norm(np.transpose(beta_exact),ord=2)
print('\nError in values of coefficients = %8.4f' %err_val_3)

## Using a sequential quadratic programming algorithm   
                   
time_start4 = time.process_time()                   

res4 = minimize(ridge_reg, beta_0, args=(X, Y, rho), method='COBYLA', options={'disp': True})
                   
time_elapsed4 = (time.process_time() - time_start4)   

print('\nValues of the least squares coefficients obtained with COBYLA for the Ridge problem:')
for i in range(K+1):
    print('beta %3d %7.3f' %(i,res.x[i]))
print('Elapsed time = %8.5f' %(time_elapsed4))

err_val_4 = np.linalg.norm(np.transpose(beta_exact)-res.x,ord=2)/np.linalg.norm(np.transpose(beta_exact),ord=2)
print('\nError in values of coefficients = %8.4f' %err_val_4)


Values of the Ridge coefficients obtained with Nelder-Mead:
beta   0  14.537
beta   1   0.934
beta   2   0.580
beta   3   0.340
beta   4   1.788
beta   5  -4.884
beta   6  -3.220
beta   7  -0.827
beta   8  -0.456
beta   9  -5.606
beta  10  -0.922
beta  11   8.543
beta  12  -0.535
beta  13   1.635
beta  14   1.486
beta  15  -0.832
beta  16  -0.736
beta  17   1.919
beta  18  -1.150
beta  19   0.051
beta  20   0.131
beta  21  -3.397
beta  22   4.323
beta  23  -7.094
beta  24  -1.911
beta  25  -0.779
beta  26  -0.779
beta  27  -4.701
beta  28   0.821
beta  29   0.710
beta  30   3.184
beta  31  -0.859
beta  32   1.788
beta  33   0.867
beta  34  -2.410
beta  35  -1.173
beta  36  -5.865
beta  37  -1.344
beta  38   2.411
beta  39  -1.041
beta  40  -0.295
beta  41   0.055
beta  42   8.598
beta  43  -0.849
beta  44  -1.757
beta  45  -0.408
beta  46   1.160
beta  47   0.725
beta  48  -1.065
beta  49   1.221
beta  50  -1.762
beta  51  -1.005
beta  52  -3.607
beta  53  -1.077
beta  54   5.110
beta

#### c) Estimate the value of the regression coefficients by implementing the: 

#### Consider a line search technique to improve the algorithm convergence, e.x., Armijo rule. Compare the performance of these algorithms (number of iterations and total computational time).

##### i. Gradient method.

Steepest descent method: We start from an initial iterate $x_0$, and compute search descent directions $p_k=-\nabla f(x_k)$. Far from the solution, compute a steplength $\alpha_k>0$
The initial iterate $x_0$ is moved the following way:
$$x_{k+1} = x_k + \alpha_k\ p_k$$ until it converges to a local solution

In [5]:
# Implementation of the gradient method
from numpy.linalg import norm

(a,b) = X.shape

## Algorithm's parameters

alpha = 1e-10 #steplenght
n_it = 50000  #number of iterations
epsilon = 1e-5 
tol = 10000 #tolerance

## Initial values for the variables and data containers

beta_gradient = np.zeros(b) 
OF_it = np.zeros(n_it)
tol_it = np.zeros(n_it)
alpha_it = np.zeros(n_it)

# We implement the gradient method

start = time.process_time()

i = 0

while (i <= n_it-2) and (tol > epsilon):
    i = i + 1
    grad = ridge_reg_der(beta_gradient,X,Y, rho) # Gradient vector
    ddirect = -grad # Descent direction
    beta_gradient = beta_gradient + alpha*ddirect
    
    OF_it[i] = ridge_reg(beta_gradient, X, Y, rho)
    tol = np.linalg.norm(grad,ord=2)
    tol_it[i] = tol
    alpha_it[i] = alpha
    
total_time = (time.process_time() - start)

print('Elapsed computational time = %8.5f' %(total_time))
print('\nNumber of iterations = %5.0f' %i)
print('Objective function   = %11.5f' %OF_it[i])
print('Optimality tolerance = %11.5f' %tol)
print('\nValues of the least squares coefficients with the gradient method:')
print('beta %-9s %7.3f' %('intercept',beta_gradient[0]))
print('beta %-9s %7.3f' %('1',beta_gradient[1]))
print('beta %-9s %7.3f' %('2',beta_gradient[2]))
print('beta %-9s %7.3f' %('3',beta_gradient[3]))
print('beta %-9s %7.3f' %('4',beta_gradient[4]))
print('beta %-9s %7.3f' %('5',beta_gradient[5]))
print('beta %-9s %7.3f' %('6',beta_gradient[6]))

beta_error = np.linalg.norm(np.transpose(beta_exact)-beta_gradient,ord=2)/np.linalg.norm(beta_gradient,ord=2)
print('\nBeta coefficient error = %10.5f' %beta_error)


Elapsed computational time = 63.64062

Number of iterations = 49999
Objective function   =  6966.46803
Optimality tolerance = 96650.90766

Values of the least squares coefficients with the gradient method:
beta intercept  -0.007
beta 1           1.007
beta 2           0.008
beta 3          -0.994
beta 4           2.982
beta 5           3.996
beta 6          -3.994

Beta coefficient error =    0.04101


##### ii. Newton method.


Start from an initial iterate $x_0$, compute a search descent direction 
$$p_k=-(\nabla^2 f(x_k))^{-1} \nabla f(x_k)$$

such that $\nabla^2 f(x_k)$ is nonsingular. Far away from the solution, compute a steplength $\alpha_k>0$

From the initial iterate, move in the following direction $$x_{k+1} = x_k + \alpha_k\ p_k$$
until it converges to a local solution.


In [22]:
(a,b) = X.shape

## Parameters 

alpha = 1e-6
n_it = 200 
epsilon = 1e-6
tol = 100
sigma = 0.2
delta = 0.1

## Initial values for the variables and data containers

beta_newton = np.zeros(b) # initial value for beta

OF_it = np.zeros(n_it)
tol_it = np.zeros(n_it)
alpha_it = np.zeros(n_it)

# Implement Newton's method

start = time.process_time()

i = 0

while (i <= n_it-2) and (tol > epsilon):
    i = i + 1
    grad = ridge_reg_der(beta_newton,X,Y, rho)
    hess = ridge_hess(beta_newton,X,Y, rho)
    ddirect = -np.dot(np.linalg.inv(hess),grad) 
    
#We use the Armijo rule to adjust alpha and improve algorithm convergence
    alpha=1
    while (ridge_reg(beta_newton+alpha*ddirect,X,Y, rho) > ridge_reg(beta_newton,X,Y, rho)+alpha*sigma*np.dot(ddirect,grad)):
        alpha = alpha*delta

    beta_newton = beta_newton + alpha*ddirect
    OF_it[i] = ridge_reg(beta_newton, X, Y, rho)
    tol = np.linalg.norm(grad,ord=2)
    tol_it[i] = tol
    alpha_it[i] = alpha

total_time = (time.process_time() - start)

## Print the results

print('Elapsed computational time = %8.5f' %(total_time))
print('\nNumber of iterations = %5.0f' %i)
print('Objective function   = %11.5f' %OF_it[i])
print('Optimality tolerance = %11.5f' %tol)

print('\nValues of the least squares coefficients:')
print('beta %-9s %7.3f' %('intercept',beta_newton[0]))
print('beta %-9s %7.3f' %('1',beta_newton[1]))
print('beta %-9s %7.3f' %('2',beta_newton[2]))
print('beta %-9s %7.3f' %('3',beta_newton[3]))
print('beta %-9s %7.3f' %('4',beta_newton[4]))
print('beta %-9s %7.3f' %('5',beta_newton[5]))
print('beta %-9s %7.3f' %('6',beta_newton[6]))

beta_error = np.linalg.norm(np.transpose(beta_exact)-beta_newton,ord=2)/np.linalg.norm(beta_newton,ord=2)
print('\nBeta coefficient error = %10.5f' %beta_error)


Elapsed computational time =  2.73438

Number of iterations =   199
Objective function   =  1837.86840
Optimality tolerance =     0.00001

Values of the least squares coefficients:
beta intercept  -1.209
beta 1           0.999
beta 2          -0.001
beta 3          -1.002
beta 4           2.999
beta 5           4.001
beta 6          -4.002

Beta coefficient error =    0.00000


##### iii. Quasi-Newton method.

Start from an initial approximation $B_0$ to $H_0 = \nabla^2 f(x_0)$ and move it as:

$$B_{k+1}=B_k+ U$$

With U an update rule with to versions:

-Symmetric rank-one: 
$$U = \frac{(y_k-B_k s_k)(y_k-B_k s_k)^T)}{(y_k-B_k s_k)^T s_k}$$

-BFGS:
$$U = \frac{(B_k s_k)(B_k s_k)^T}{s_k^T B_k s_k}+\frac{y_k y_k^T}{y_k^T s_k}$$

with $s_k = x_{k+1}-x_k$, $y_k = \nabla f(x_{k+1})-\nabla f(x_{k})$



In [161]:
(a,b) = X.shape

## Parameters 

alpha = 1e-3
n_it = 101
epsilon = 1e-7
tol = 100
sigma = 0.2
delta = 0.1

## Initial values for the variables and data containers

beta_quasi = np.zeros(b) # initial value for beta

OF_it = np.zeros(n_it)
tol_it = np.zeros(n_it)
alpha_it = np.zeros(n_it)
s = np.zeros(b)
yk = np.zeros(n_it)
h = np.zeros(n_it)
t = np.zeros(n_it)
r = np.zeros(n_it)
u = np.zeros(n_it)


# Implement Quasi-Newton's method with the symmetric rank-one update rule:

start = time.process_time()

i = 0

for i in range(1, len(beta_quasi)):
    if tol <= epsilon:
        break
#while (i <= n_it-2) and (tol > epsilon):
        #i = i + 1
        grad = ridge_reg_der(beta_quasi,X,Y, rho)
        hess = ridge_hess(beta_quasi,X,Y, rho)
        ddirect = -np.dot(np.linalg.inv(hess),grad)
        
        alpha=1
    while (ridge_reg(beta_newton+alpha*ddirect,X,Y, rho) > ridge_reg(beta_newton,X,Y, rho)+alpha*sigma*np.dot(ddirect,grad)):
        alpha = alpha*delta
        
        #beta_quasi = beta_quasi + alpha*ddirect
        s[i] = beta_quasi[i]-beta_quasi[i-1]
        yk = grad[i]-grad[i-1]
        h = np.dot(hess,s)
        t = np.dot((yk-h),(yk-h).T)
        r = np.power(np.dot(((yk-h).T),s),-1)
        u = np.dot(t,r)

        hess[i] = hess[i-1] + u
        beta_quasi = beta_quasi + alpha*ddirect
        OF_it[i] = ridge_reg(beta_quasi, X, Y, rho)
        tol = np.linalg.norm(grad,ord=2)
        tol_it[i] = tol
        alpha_it[i] = alpha

total_time = (time.process_time() - start)    
print('Elapsed computational time = %8.5f' %(total_time))
print('\nNumber of iterations = %5.0f' %i)
print('Objective function   = %11.5f' %OF_it[i])
print('Optimality tolerance = %11.5f' %tol)
print('\nValues of the least squares coefficients:')
print('beta %-9s %7.3f' %('intercept',beta_quasi[0]))
print('beta %-9s %7.3f' %('1',beta_quasi[1]))
print('beta %-9s %7.3f' %('2',beta_quasi[2]))
print('beta %-9s %7.3f' %('3',beta_quasi[3]))
print('beta %-9s %7.3f' %('4',beta_quasi[4]))
print('beta %-9s %7.3f' %('5',beta_quasi[5]))
print('beta %-9s %7.3f' %('6',beta_quasi[6]))

beta_error = np.linalg.norm(np.transpose(beta_exact)-beta_quasi,ord=2)/np.linalg.norm(beta_quasi,ord=2)
print('\nBeta coefficient error = %10.5f' %beta_error)


Elapsed computational time =  0.12500

Number of iterations =   100
Objective function   =     0.00000
Optimality tolerance = 41069956.90801

Values of the least squares coefficients:
beta intercept  -0.000
beta 1           2.562
beta 2           0.916
beta 3           0.170
beta 4           5.168
beta 5           6.217
beta 6          -4.386

Beta coefficient error =    0.40264


#### d) Estimate the value of the regression coefficients by implementing the:



##### i. Coordinate gradient method.


Just modify $x_i$ to improve the objective function and:

1: Choose an index $i_k \in \{1,2,\dots,p\}$ randomly

2: $x^{k+1}=x^{k}-\alpha\nabla_{i_{k}}f(x^{k})e_{ik}$

where $e_i$ is the $i^\text{th}$ canonical coordinate vector and $\nabla_{i}f(x)$
is the $i^\text{th}$ coordinate of the gradient.

In [6]:
#First, define the coordinated gradient
def ridge_reg(beta_ridge,X,Y,rho):
    beta_ridge = np.matrix(beta_ridge)
    z = Y - np.dot(X,beta_ridge.T)
    val = np.dot(z.T,z) + rho*np.dot(beta_ridge,beta_ridge.T)
    return val 

def ridge_der_coord(beta_ridge,index,X,Y):
    beta_ridge = np.matrix(beta_ridge)
    pp=-2*np.dot((Y-np.dot(X,beta_ridge)).T,X[:,index])
    #pp=np.array(-2*np.dot((Y-np.dot(X,beta_ls)).T,X[:,index]))
    aa=np.zeros([b,1])
    aa[index]=pp
    return aa

#Define the parameters

(a,b)=X.shape
beta_ridge=np.zeros([b,1])
alpha=1e-7
n_it=50000
OF_it=np.zeros(n_it)
i=0;
tol=100;
epsilon=1e-6;

#And apply the algorithm

while (i <= n_it-2):
    i=i+1
    k = np.random.randint(b)
    grad_k = ridge_der_coord(beta_ridge,k,X,Y)
    ddirect=-grad_k
    beta_ridge=beta_ridge+alpha*ddirect
    #OF_it[i]=ridge_reg(beta_ridge, X, Y)
print('Number of iterations',i)
#print(OF_it[i])
print(beta_ridge)
print('Tolerance=',tol)
print('Error=',np.linalg.norm(beta_exact-beta_ridge,ord=2)/np.linalg.norm(beta_ridge,ord=2))

Number of iterations 49999
[[ 8.40844682e-03]
 [ 9.98630938e-01]
 [-1.70836957e-03]
 [-1.00212784e+00]
 [ 2.99903849e+00]
 [ 4.00045260e+00]
 [-4.00187690e+00]
 [ 1.99942507e+00]
 [ 3.99883564e+00]
 [ 9.98227525e-01]
 [ 3.00034099e+00]
 [-4.99845997e+00]
 [-1.18891025e-04]
 [-5.00001329e+00]
 [ 3.99957261e+00]
 [ 1.00048692e+00]
 [-3.00119243e+00]
 [-5.00167722e+00]
 [-1.10884056e-03]
 [-2.99967869e+00]
 [ 1.00093688e+00]
 [-1.99831956e+00]
 [ 1.99987384e+00]
 [-5.00042780e+00]
 [ 3.99841544e+00]
 [-5.00003133e+00]
 [-2.00065150e+00]
 [-3.00257748e+00]
 [-1.99901673e+00]
 [-3.99845338e+00]
 [-2.00087486e+00]
 [-4.00027867e+00]
 [-2.00146812e+00]
 [ 1.99952157e+00]
 [-4.00024980e+00]
 [ 2.00039452e+00]
 [-1.00011054e+00]
 [-5.00121837e+00]
 [ 2.11951250e-03]
 [-3.99951275e+00]
 [-1.70055504e-03]
 [ 4.00081244e+00]
 [ 3.99915665e+00]
 [-1.00115250e+00]
 [-4.99972763e+00]
 [ 3.99974787e+00]
 [ 2.99907147e+00]
 [ 2.99824275e+00]
 [ 9.99087570e-01]
 [ 3.00016717e+00]
 [ 1.00088857e+00]
 [-1

##### ii. Mini-batch gradient method. Study how the mini-batch size may impact the algorithm performance (number of iterations and computational time needed to reach a pre-specified tolerance limit).

Consider at each step a sample of size $b$, and iterate in $k$ selecting a random sample $(x_i, y_i)$, $i \in S_k$, and a gradient as

$$g_k \approx \frac{1}{b} \sum_{i\in S_k} \nabla F(w_k;(x_i, y_i))$$

Commonly, mini-batch sizes, $b=|S_k|$, range between 50 and 1000

In [116]:
#Define the parameters

(a,b)=X1.shape
alpha=1e-7
n_it=50000
tol=100;
epsilon=1e-6;

 
tol_it = np.zeros(n_it)
alpha_it = np.zeros(n_it)
OF_it=np.zeros(n_it)

X1 = X_s[0:200,:]
X2 = X_s[201:400,:]
X3 = X_s[401:600,:]
X4 = X_s[601:800,:]
X5 = X_s[801:1000,:] 
Y1 = Y_s[0:200,:]
Y2 = Y_s[201:400,:]
Y3 = Y_s[401:600,:]
Y4 = Y_s[601:800,:]
Y5 = Y_s[801:1000,:]
(a,b)=X1.shape #200x101 matrix
h = len(X1)
beta_batch = np.zeros(b)

P = np.concatenate([X1, X2, X3, X4, X5], axis=0)
Q = np.concatenate([Y1, Y2, Y3, Y4, Y5], axis=0)

# We implement the method

start = time.process_time()

i = 0

    

while (i <= n_it-2) and (tol > epsilon):
    i = i + 1
    grad = ridge_reg_der(beta_batch,P,Q, rho) # Gradient vector
    ddirect = -grad/b # Descent direction
    #g = grad/b 
    beta_batch = beta_batch + alpha*ddirect
    
    OF_it[i] = ridge_reg(beta_batch, P, Q, rho)
    tol = np.linalg.norm(grad,ord=2)
    tol_it[i] = tol
    alpha_it[i] = alpha
    
total_time = (time.process_time() - start)


print('Elapsed computational time = %8.5f' %(total_time))
print('\nNumber of iterations = %5.0f' %i)
print('Objective function   = %11.5f' %OF_it[i])
print('Optimality tolerance = %11.5f' %tol)
print('\nValues of the least squares coefficients with the mini-batch method:')
print('beta %-9s %7.3f' %('intercept',beta_batch[0]))
print('beta %-9s %7.3f' %('1',beta_batch[1]))
print('beta %-9s %7.3f' %('2',beta_batch[2]))
print('beta %-9s %7.3f' %('3',beta_batch[3]))
print('beta %-9s %7.3f' %('4',beta_batch[4]))
print('beta %-9s %7.3f' %('5',beta_batch[5]))
print('beta %-9s %7.3f' %('6',beta_batch[6]))

beta_error = np.linalg.norm(np.transpose(beta_exact)-beta_batch,ord=2)/np.linalg.norm(beta_batch,ord=2)
print('\nBeta coefficient error = %10.5f' %beta_error)


Elapsed computational time = 57.25000

Number of iterations = 49999
Objective function   =  1840.71280
Optimality tolerance =     9.47814

Values of the least squares coefficients with the mini-batch method:
beta intercept  -0.008
beta 1           0.999
beta 2          -0.002
beta 3          -1.002
beta 4           2.999
beta 5           4.000
beta 6          -4.002

Beta coefficient error =    0.04077


##### iii. Mini-batch gradient with momentum.

In this case, a moment is added in order to add information from past iterations:

$$v_tx_t←\beta v_t−1+g_{t,t-1}$$
$$x_t←x_{t−1}−\eta_tv_t$$

In [7]:
#Define the parameters

#(a,b)=X1.shape
alpha=1e-7
n_it=101
tol=100;
epsilon=1e-7;
eta=1e-10
 
tol_it = np.zeros(n_it)
alpha_it = np.zeros(n_it)
OF_it=np.zeros(n_it)

X1 = X_s[0:50,:]
X2 = X_s[51:100,:]
X3 = X_s[101:150,:]
X4 = X_s[151:200,:]
X5 = X_s[201:250,:]
X6 = X_s[251:300,:]
X7 = X_s[301:350,:]
X8 = X_s[351:400,:]
X9 = X_s[401:450,:]
X10 = X_s[451:500,:]
X11 = X_s[501:550,:]
X12 = X_s[551:600,:]
X13 = X_s[601:650,:]
X14 = X_s[651:700,:]
X15 = X_s[701:750,:]
X16 = X_s[751:800,:]
X17 = X_s[801:850,:]
X18 = X_s[851:900,:]
X19 = X_s[901:950,:]
X20 = X_s[951:1000,:]

Y1 = Y_s[0:50,:]
Y2 = Y_s[51:100,:]
Y3 = Y_s[101:150,:]
Y4 = Y_s[151:200,:]
Y5 = Y_s[201:250,:]
Y6 = Y_s[251:300,:]
Y7 = Y_s[301:350,:]
Y8 = Y_s[351:400,:]
Y9 = Y_s[401:450,:]
Y10 = Y_s[451:500,:]
Y11 = Y_s[501:550,:]
Y12 = Y_s[551:600,:]
Y13 = Y_s[601:650,:]
Y14 = Y_s[651:700,:]
Y15 = Y_s[701:750,:]
Y16 = Y_s[751:800,:]
Y17 = Y_s[801:850,:]
Y18 = Y_s[851:900,:]
Y19 = Y_s[901:950,:]
Y20 = Y_s[951:1000,:]

h = len(X1)
beta_momentum = np.zeros(b)

P = np.concatenate([X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20], axis=0)
Q = np.concatenate([Y1, Y2, Y3, Y4, Y5, Y6, Y7, Y8, Y9, Y10, Y11, Y12, Y13, Y14, Y15, Y16, Y17, Y18, Y19, Y20], axis=0)

start = time.process_time()

i = 0
    
while (i <= n_it-2) and (tol > epsilon):
    i=i+1
    grad = ridge_reg_der(beta_momentum,P,Q, rho) # Gradient vector
   # grad[i] = grad[i] + eta*grad[i-1]
    ddirect = -grad/b # Descent direction 
    beta_momentum = beta_momentum + alpha*ddirect
    grad[i] = grad[i] + eta*grad[i-1]

    OF_it[i] = ridge_reg(beta_momentum, P, Q, rho)
    tol = np.linalg.norm(grad,ord=2)
    tol_it[i] = tol
    alpha_it[i] = alpha
    
total_time = (time.process_time() - start)

print('Elapsed computational time = %8.5f' %(total_time))
print('\nNumber of iterations = %5.0f' %i)
print('Objective function   = %11.5f' %OF_it[i])
print('Optimality tolerance = %11.5f' %tol)
print('\nValues of the least squares coefficients with the mini-batch method with momentum:')
print('beta %-9s %7.3f' %('intercept',beta_momentum[0]))
print('beta %-9s %7.3f' %('1',beta_momentum[1]))
print('beta %-9s %7.3f' %('2',beta_momentum[2]))
print('beta %-9s %7.3f' %('3',beta_momentum[3]))
print('beta %-9s %7.3f' %('4',beta_momentum[4]))
print('beta %-9s %7.3f' %('5',beta_momentum[5]))
print('beta %-9s %7.3f' %('6',beta_momentum[6]))

beta_error = np.linalg.norm(np.transpose(beta_exact)-beta_momentum,ord=2)/np.linalg.norm(beta_momentum,ord=2)
print('\nBeta coefficient error = %10.5f' %beta_error)

Elapsed computational time =  0.18750

Number of iterations =   100
Objective function   = 492332410.66486
Optimality tolerance = 40899726.99627

Values of the least squares coefficients with the mini-batch method with momentum:
beta intercept  -0.007
beta 1          -0.106
beta 2          -0.268
beta 3          -0.330
beta 4           0.157
beta 5           0.256
beta 6          -0.780

Beta coefficient error =    4.38301


### Conclusions
From the results derived previously, it can be infered that by using unconstrained methods, the errors are considerably smaller than the ones regarding constrained methods (errors of about 0.04), except from the last method, the mini-batch with momentum. This can be due to the number of samples in which the dataset has been divided, as it does not seem to be the optimal one. Nevertheless, for instance, no error is shown with the Newton method, showing perfect accuracy (good accuracy was expected), and it can be concluded that the goal was fulfilled, since the errors were dropped to negligible values starting with constrained methods.